In [13]:
#import libaries
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model

## upload Jordan Clarkson 2021 data

address = '/Users/raytrey5/Desktop/IST_718_Project/Joe_Ingles_2021_GameLog.csv'
Joe_df = pd.read_csv(address)
## drop NAs columns 
Joe_df = Joe_df.drop(columns=Joe_df.columns[24:30])
pos = [64,65,66,67,68]
Joe_df.drop(Joe_df.index[pos], inplace=True)
Joe_df

,HomeAway,Opp,WinLoss,GS,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Three+3s
0,A,ATL,W,0.0,27.0,3.0,5.0,0.600,0.0,1.0,...,0.0,4.0,4.0,4.0,1.0,0.0,0.0,2.0,6.0,0.0
1,A,BOS,W,0.0,25.0,5.0,9.0,0.556,4.0,8.0,...,0.0,5.0,5.0,2.0,1.0,0.0,0.0,2.0,14.0,1.0
2,H,BOS,W,1.0,36.0,5.0,12.0,0.417,5.0,11.0,...,0.0,3.0,3.0,6.0,0.0,0.0,1.0,3.0,24.0,1.0
3,H,BRK,W,0.0,26.0,1.0,6.0,0.167,1.0,6.0,...,0.0,7.0,7.0,3.0,1.0,0.0,1.0,2.0,3.0,0.0
4,A,BRK,L,0.0,23.0,2.0,6.0,0.333,1.0,2.0,...,0.0,3.0,3.0,2.0,0.0,0.0,2.0,3.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,A,SAS,W,0.0,21.0,5.0,9.0,0.556,2.0,6.0,...,2.0,5.0,7.0,1.0,0.0,0.0,0.0,1.0,13.0,0.0
60,H,TOR,W,1.0,34.0,4.0,8.0,0.500,4.0,7.0,...,0.0,3.0,3.0,9.0,0.0,0.0,4.0,3.0,15.0,1.0
61,A,TOR,W,0.0,33.0,5.0,6.0,0.833,5.0,6.0,...,0.0,5.0,5.0,6.0,1.0,0.0,1.0,1.0,19.0,1.0
62,H,WAS,L,1.0,31.0,6.0,16.0,0.375,3.0,9.0,...,2.0,2.0,4.0,6.0,0.0,1.0,3.0,3.0,18.0,1.0


In [14]:
# Fill in Nas with mean... Free throw percentage...There were
# games he did not attempt a Free throw thus it was NA

column_means = Joe_df.mean()
Joe_df = Joe_df.fillna(column_means)

In [19]:
## check to see if there's no NAs
Joe_df.isnull().values.any()

False

In [20]:
## upload 2021 Defensive metrics

address = '/Users/raytrey5/Desktop/IST_718_Project/OpponentStatsPerGame2021.csv'
Defense2021 = pd.read_csv(address)
Defense2021 = Defense2021.dropna()
Defense2021

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,Atlanta Hawks,41.4,88.9,0.466,11.9,33.8,0.351,29.6,55.1,0.537,...,12.3,111.8,35.1,24.1,7.0,4.8,0.530,11.1,77.8,0.190
1,Boston Celtics,40.1,86.0,0.466,13.0,34.6,0.377,27.1,51.5,0.526,...,14.1,111.6,33.6,23.5,7.7,5.3,0.539,12.8,78.1,0.215
2,Brooklyn Nets,42.1,91.1,0.461,13.3,36.2,0.369,28.7,55.0,0.523,...,12.7,114.4,35.5,26.8,6.7,5.3,0.531,11.1,77.3,0.187
3,Charlotte Hornets,41.2,88.3,0.467,14.4,39.6,0.364,26.8,48.7,0.550,...,14.8,111.2,33.2,26.8,7.8,4.8,0.550,13.2,75.7,0.163
4,Chicago Bulls,41.5,87.6,0.474,11.5,32.2,0.356,30.0,55.4,0.542,...,12.8,111.8,35.3,26.8,6.7,4.2,0.538,11.6,80.3,0.196
5,Cleveland Cavaliers,42.0,86.6,0.486,12.4,32.4,0.383,29.6,54.1,0.547,...,14.5,112.3,32.3,23.8,7.8,4.5,0.556,13.0,76.6,0.183
6,Dallas Mavericks,40.0,87.1,0.459,12.9,35.2,0.365,27.1,51.8,0.523,...,12.7,109.9,34.2,22.9,6.3,4.3,0.534,11.5,77.8,0.197
7,Denver Nuggets,40.1,85.5,0.469,13.0,35.5,0.365,27.2,50.0,0.543,...,14.1,110.2,33.9,26.8,8.1,4.5,0.545,12.9,78.9,0.200
8,Detroit Pistons,41.3,87.0,0.475,11.4,31.3,0.362,29.9,55.7,0.538,...,14.0,111.1,33.1,24.2,7.4,5.2,0.541,12.6,76.9,0.195
9,Golden State Warriors,40.2,88.8,0.452,12.5,34.7,0.360,27.7,54.1,0.511,...,15.2,112.7,35.1,27.7,8.2,4.8,0.522,13.1,76.6,0.223


In [21]:
## Duplicate the amount of times Clarkson opponents then delete his team and if he plays the team less than 3 times
newDefense2021 = pd.DataFrame(np.repeat(Defense2021.values,4,axis=0))
newDefense2021.columns = Defense2021.columns
print(newDefense2021)

                   Team OppFG OppFGA OppFGP Opp3P Opp3PA Opp3PPerc Opp2P  \
0         Atlanta Hawks  41.4   88.9  0.466  11.9   33.8     0.351  29.6   
1         Atlanta Hawks  41.4   88.9  0.466  11.9   33.8     0.351  29.6   
2         Atlanta Hawks  41.4   88.9  0.466  11.9   33.8     0.351  29.6   
3         Atlanta Hawks  41.4   88.9  0.466  11.9   33.8     0.351  29.6   
4        Boston Celtics  40.1     86  0.466    13   34.6     0.377  27.1   
..                  ...   ...    ...    ...   ...    ...       ...   ...   
115           Utah Jazz  40.9   91.5  0.447  10.9   32.1     0.341    30   
116  Washington Wizards  43.2   91.7  0.471  12.5   33.5     0.373  30.7   
117  Washington Wizards  43.2   91.7  0.471  12.5   33.5     0.373  30.7   
118  Washington Wizards  43.2   91.7  0.471  12.5   33.5     0.373  30.7   
119  Washington Wizards  43.2   91.7  0.471  12.5   33.5     0.373  30.7   

    Opp2PA Opp2PPerc  ... OppTOV OppPTS Team_DRB Team_AST Team_STL Team_BLK  \
0     55

In [34]:
print(Joe_df[50:])

   HomeAway  Opp WinLoss   GS    MP    FG   FGA    FG%   3P   3PA  ...  ORB  \
50        A  PHO       L  1.0  29.0   5.0  14.0  0.357  1.0   7.0  ...  2.0   
51        A  PHO       L  0.0  27.0   2.0   4.0  0.500  1.0   2.0  ...  0.0   
52        H  PHO       L  0.0  32.0   2.0   7.0  0.286  1.0   5.0  ...  1.0   
53        H  POR       W  0.0  35.0   4.0  13.0  0.308  2.0   8.0  ...  0.0   
54        A  POR       W  0.0  27.0   5.0  10.0  0.500  3.0   8.0  ...  0.0   
55        A  SAC       W  1.0  24.0   4.0   7.0  0.571  3.0   6.0  ...  0.0   
56        H  SAC       W  0.0  31.0   6.0  11.0  0.545  5.0  10.0  ...  0.0   
57        H  SAS       W  1.0  25.0   5.0  11.0  0.455  3.0   8.0  ...  0.0   
58        H  SAS       W  1.0  31.0   5.0  11.0  0.455  2.0   7.0  ...  1.0   
59        A  SAS       W  0.0  21.0   5.0   9.0  0.556  2.0   6.0  ...  2.0   
60        H  TOR       W  1.0  34.0   4.0   8.0  0.500  4.0   7.0  ...  0.0   
61        A  TOR       W  0.0  33.0   5.0   6.0  0.8

In [22]:
### Filter by opponent then combine after getting the correct amount of duplicates
df_Hawks = newDefense2021[newDefense2021['Team'] == 'Atlanta Hawks']
df_Hawks = df_Hawks.drop(df_Hawks.index[[[1,2,3]]])
df_Hawks

/Users/raytrey5/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:4291: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,Atlanta Hawks,41.4,88.9,0.466,11.9,33.8,0.351,29.6,55.1,0.537,...,12.3,111.8,35.1,24.1,7,4.8,0.53,11.1,77.8,0.19


In [24]:
df_Celtics = newDefense2021[newDefense2021['Team'] == 'Boston Celtics']
df_Celtics = df_Celtics.drop(df_Celtics.index[[2,3]])
df_Celtics

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
4,Boston Celtics,40.1,86,0.466,13,34.6,0.377,27.1,51.5,0.526,...,14.1,111.6,33.6,23.5,7.7,5.3,0.539,12.8,78.1,0.215
5,Boston Celtics,40.1,86,0.466,13,34.6,0.377,27.1,51.5,0.526,...,14.1,111.6,33.6,23.5,7.7,5.3,0.539,12.8,78.1,0.215


In [25]:
df_Nets = newDefense2021[newDefense2021['Team'] == 'Brooklyn Nets']
df_Nets = df_Nets.drop(df_Nets.index[[2,3]])
df_Nets

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
8,Brooklyn Nets,42.1,91.1,0.461,13.3,36.2,0.369,28.7,55,0.523,...,12.7,114.4,35.5,26.8,6.7,5.3,0.531,11.1,77.3,0.187
9,Brooklyn Nets,42.1,91.1,0.461,13.3,36.2,0.369,28.7,55,0.523,...,12.7,114.4,35.5,26.8,6.7,5.3,0.531,11.1,77.3,0.187


In [26]:
df_Bulls = newDefense2021[newDefense2021['Team'] == 'Chicago Bulls']
df_Bulls = df_Bulls.drop(df_Bulls.index[[2,3]])
df_Bulls

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
16,Chicago Bulls,41.5,87.6,0.474,11.5,32.2,0.356,30,55.4,0.542,...,12.8,111.8,35.3,26.8,6.7,4.2,0.538,11.6,80.3,0.196
17,Chicago Bulls,41.5,87.6,0.474,11.5,32.2,0.356,30,55.4,0.542,...,12.8,111.8,35.3,26.8,6.7,4.2,0.538,11.6,80.3,0.196


In [27]:
df_Hornets = newDefense2021[newDefense2021['Team'] == 'Charlotte Hornets']
df_Hornets = df_Hornets.drop(df_Hornets.index[[2,3]])
df_Hornets

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
12,Charlotte Hornets,41.2,88.3,0.467,14.4,39.6,0.364,26.8,48.7,0.55,...,14.8,111.2,33.2,26.8,7.8,4.8,0.55,13.2,75.7,0.163
13,Charlotte Hornets,41.2,88.3,0.467,14.4,39.6,0.364,26.8,48.7,0.55,...,14.8,111.2,33.2,26.8,7.8,4.8,0.55,13.2,75.7,0.163


In [28]:
df_Cavs = newDefense2021[newDefense2021['Team'] == 'Cleveland Cavaliers']
df_Cavs = df_Cavs.drop(df_Cavs.index[[[1,2,3]]])
df_Cavs

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
20,Cleveland Cavaliers,42,86.6,0.486,12.4,32.4,0.383,29.6,54.1,0.547,...,14.5,112.3,32.3,23.8,7.8,4.5,0.556,13,76.6,0.183


In [29]:
df_Mavs = newDefense2021[newDefense2021['Team'] == 'Dallas Mavericks']
df_Mavs = df_Mavs.drop(df_Mavs.index[1])
df_Mavs

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
24,Dallas Mavericks,40,87.1,0.459,12.9,35.2,0.365,27.1,51.8,0.523,...,12.7,109.9,34.2,22.9,6.3,4.3,0.534,11.5,77.8,0.197
26,Dallas Mavericks,40,87.1,0.459,12.9,35.2,0.365,27.1,51.8,0.523,...,12.7,109.9,34.2,22.9,6.3,4.3,0.534,11.5,77.8,0.197
27,Dallas Mavericks,40,87.1,0.459,12.9,35.2,0.365,27.1,51.8,0.523,...,12.7,109.9,34.2,22.9,6.3,4.3,0.534,11.5,77.8,0.197


In [30]:
df_Nuggets = newDefense2021[newDefense2021['Team'] == 'Denver Nuggets']
df_Nuggets = df_Nuggets.drop(df_Nuggets.index[[1,2]])

In [31]:
df_Pistons = newDefense2021[newDefense2021['Team'] == 'Detroit Pistons']
df_Pistons = df_Pistons.drop(df_Pistons.index[[2,3]])

In [32]:
df_Warriors = newDefense2021[newDefense2021['Team'] == 'Golden State Warriors']
df_Warriors = df_Warriors.drop(df_Warriors.index[1])

In [33]:
df_Rockets = newDefense2021[newDefense2021['Team'] == 'Houston Rockets']
df_Rockets = df_Rockets.drop(df_Rockets.index[1])

In [35]:
df_Pacers = newDefense2021[newDefense2021['Team'] == 'Indiana Pacers']
df_Pacers = df_Pacers.drop(df_Pacers.index[[2,3]])
df_Pacers

df_Clippers = newDefense2021[newDefense2021['Team'] == 'Los Angeles Clippers*']
df_Clippers = df_Clippers.drop(df_Clippers.index[1])

df_Lakers = newDefense2021[newDefense2021['Team'] == 'Los Angeles Lakers']
df_Lakers = df_Lakers.drop(df_Lakers.index[1])

df_Grizz = newDefense2021[newDefense2021['Team'] == 'Memphis Grizzlies']
df_Grizz = df_Grizz.drop(df_Grizz.index[1])

df_Heat = newDefense2021[newDefense2021['Team'] == 'Miami Heat']
df_Heat = df_Heat.drop(df_Heat.index[[2,3]])

df_Bucks = newDefense2021[newDefense2021['Team'] == 'Milwaukee Bucks']
df_Bucks = df_Bucks.drop(df_Bucks.index[2])

df_Wolves = newDefense2021[newDefense2021['Team'] == 'Minnesota Timberwolves']
df_Wolves = df_Wolves.drop(df_Wolves.index[2])

df_Pels = newDefense2021[newDefense2021['Team'] == 'New Orleans Pelicans']
df_Pels = df_Pels.drop(df_Pels.index[2])

df_Knicks = newDefense2021[newDefense2021['Team'] == 'New York Knicks']
df_Knicks = df_Knicks.drop(df_Knicks.index[[2,3]])

df_Thunder = newDefense2021[newDefense2021['Team'] == 'Oklahoma City Thunder']
df_Thunder = df_Thunder.drop(df_Thunder.index[[[1,2,3]]])

df_Magic = newDefense2021[newDefense2021['Team'] == 'Orlando Magic']
df_Magic = df_Magic.drop(df_Magic.index[[2,3]])

df_Sixers = newDefense2021[newDefense2021['Team'] == 'Philadelphia 76ers']
df_Sixers = df_Sixers.drop(df_Sixers.index[[2,3]])

df_Suns = newDefense2021[newDefense2021['Team'] == 'Phoenix Suns']
df_Suns = df_Suns.drop(df_Suns.index[2])

df_Blazers = newDefense2021[newDefense2021['Team'] == 'Portland Trail Blazers']
df_Blazers = df_Blazers.drop(df_Blazers.index[[1,2]])

df_Kings = newDefense2021[newDefense2021['Team'] == 'Sacramento Kings']
df_Kings = df_Kings.drop(df_Kings.index[[1,2]])

df_Spurs = newDefense2021[newDefense2021['Team'] == 'San Antonio Spurs']
df_Spurs = df_Spurs.drop(df_Spurs.index[2])

df_Raptors = newDefense2021[newDefense2021['Team'] == 'Toronto Raptors']
df_Raptors = df_Raptors.drop(df_Raptors.index[[1,2]])

df_Wiz = newDefense2021[newDefense2021['Team'] == 'Washington Wizards']
df_Wiz = df_Wiz.drop(df_Wiz.index[[1,2]])


In [36]:
frames = [df_Hawks, df_Celtics, df_Nets, df_Bulls, df_Hornets, df_Cavs, df_Mavs, df_Nuggets, df_Pistons, df_Warriors,
         df_Rockets, df_Pacers, df_Clippers, df_Lakers, df_Grizz, df_Heat, df_Bucks, df_Wolves, df_Pels,
         df_Knicks, df_Thunder, df_Magic, df_Sixers, df_Suns, df_Blazers, df_Kings, df_Spurs, df_Raptors, df_Wiz ]

FinalDefense2021 = pd.concat(frames)

In [38]:
## Combine Jingles Gamelog with Defensive metrics

Joe_df2021 = pd.concat([Joe_df.reset_index(drop=True),FinalDefense2021.reset_index(drop=True)], axis=1)
Joe_df2021 = Joe_df2021.dropna()
Joe_df2021

,HomeAway,Opp,WinLoss,GS,MP,FG,FGA,FG%,3P,3PA,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,A,ATL,W,0.0,27.0,3.0,5.0,0.600,0.0,1.0,...,12.3,111.8,35.1,24.1,7,4.8,0.53,11.1,77.8,0.19
1,A,BOS,W,0.0,25.0,5.0,9.0,0.556,4.0,8.0,...,14.1,111.6,33.6,23.5,7.7,5.3,0.539,12.8,78.1,0.215
2,H,BOS,W,1.0,36.0,5.0,12.0,0.417,5.0,11.0,...,14.1,111.6,33.6,23.5,7.7,5.3,0.539,12.8,78.1,0.215
3,H,BRK,W,0.0,26.0,1.0,6.0,0.167,1.0,6.0,...,12.7,114.4,35.5,26.8,6.7,5.3,0.531,11.1,77.3,0.187
4,A,BRK,L,0.0,23.0,2.0,6.0,0.333,1.0,2.0,...,12.7,114.4,35.5,26.8,6.7,5.3,0.531,11.1,77.3,0.187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,A,SAS,W,0.0,21.0,5.0,9.0,0.556,2.0,6.0,...,13.4,112.2,34.6,24.4,7,5.1,0.541,11.8,77.3,0.174
60,H,TOR,W,1.0,34.0,4.0,8.0,0.500,4.0,7.0,...,13.4,112.2,34.6,24.4,7,5.1,0.541,11.8,77.3,0.174
61,A,TOR,W,0.0,33.0,5.0,6.0,0.833,5.0,6.0,...,13.4,112.2,34.6,24.4,7,5.1,0.541,11.8,77.3,0.174
62,H,WAS,L,1.0,31.0,6.0,16.0,0.375,3.0,9.0,...,16.2,111.5,32.1,24.1,8.6,5.4,0.543,14.4,76.3,0.234


In [41]:
### Repeat for previous 2 seasons for the training data

## upload Joe Ingles 2020 data

address = '/Users/raytrey5/Desktop/IST_718_Project/Joe_Ingles_2020_GameLog.csv'
Joe2020 = pd.read_csv(address)
## drop NAs columns 
## drop NAs columns 
Joe2020 = Joe2020.drop(columns=Joe2020.columns[24:30])
Joe2020

,HomeAway,Opp,WinLoss,GS,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Three+3s
0,A,ATL,W,1,36,5,10,0.500,2,5,...,4,3,7,5,2,0,3,4,14,0
1,A,BOS,W,0,30,3,7,0.429,3,6,...,1,6,7,6,3,0,1,1,9,1
2,H,BOS,L,0,26,1,3,0.333,0,2,...,0,2,2,2,0,1,0,2,2,0
3,A,BRK,W,1,30,10,14,0.714,6,8,...,0,3,3,5,1,0,1,0,27,1
4,H,BRK,W,0,32,0,6,0.000,0,4,...,0,5,5,2,0,1,1,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,A,SAS,L,1,30,3,8,0.375,1,4,...,1,1,2,4,0,0,0,2,7,0
68,H,TOR,L,0,32,8,12,0.667,4,8,...,1,4,5,6,2,0,3,1,20,1
69,A,TOR,L,0,26,2,5,0.400,2,3,...,0,3,3,4,0,0,0,1,6,0
70,H,WAS,W,0,27,3,5,0.600,2,4,...,1,6,7,6,0,0,2,2,10,0


In [58]:
print(Joe2020[0:49])

   HomeAway  Opp WinLoss  GS  MP  FG  FGA    FG%  3P  3PA  ...  ORB  DRB  TRB  \
0         A  ATL       W   1  36   5   10  0.500   2    5  ...    4    3    7   
1         A  BOS       W   0  30   3    7  0.429   3    6  ...    1    6    7   
2         H  BOS       L   0  26   1    3  0.333   0    2  ...    0    2    2   
3         A  BRK       W   1  30  10   14  0.714   6    8  ...    0    3    3   
4         H  BRK       W   0  32   0    6  0.000   0    4  ...    0    5    5   
5         A  CHI       W   1  34   3    9  0.333   2    5  ...    0    5    5   
6         H  CHO       W   1  21   3    6  0.500   3    5  ...    0    0    0   
7         A  CHO       W   1  39   5   10  0.500   4    8  ...    0    2    2   
8         A  CLE       W   0  29   1    2  0.500   1    2  ...    0    3    3   
9         H  DAL       L   1  13   3    7  0.429   1    5  ...    0    4    4   
10        A  DAL       W   1  32   2    8  0.250   1    7  ...    0    3    3   
11        H  DAL       W   1

In [49]:
print(Joe2020[49:])

   HomeAway  Opp WinLoss  GS  MP  FG  FGA    FG%  3P  3PA  ...  ORB  DRB  TRB  \
49        A  OKC       L   1  32   3    6  0.500   1    4  ...    1    1    2   
50        H  OKC       L   1  34   3    7  0.429   3    6  ...    0    4    4   
51        H  OKC       W   0  33   3    8  0.375   1    4  ...    0    4    4   
52        A  ORL       W   1  33   4   11  0.364   3    9  ...    0    4    4   
53        H  ORL       W   1  37   5    8  0.625   4    4  ...    2   10   12   
54        A  PHI       L   0  31   5   10  0.500   2    4  ...    0    8    8   
55        H  PHI       W   0  29   6   11  0.545   2    5  ...    1    1    2   
56        H  PHO       L   1  29   2    7  0.286   2    5  ...    0    0    0   
57        A  PHO       W   0  26   1    6  0.167   1    5  ...    1    8    9   
58        H  POR       W   1  33   5   10  0.500   3    5  ...    1    5    6   
59        A  POR       L   1  28   1    2  0.500   1    2  ...    0    1    1   
60        H  POR       W   1

In [42]:
# Fill in Nas with mean... Free throw percentage...There were
# games he did not attempt a Free throw thus it was NA

column_means = Joe2020.mean()
Joe2020 = Joe2020.fillna(column_means)
## check to see if there's no NAs
Joe2020.isnull().values.any()

False

In [43]:
## upload 2020 Defensive metrics

address = '/Users/raytrey5/Desktop/IST_718_Project/OpponentStatsPerGame2020.csv'
Defense2020 = pd.read_csv(address)
Defense2020 = Defense2020.dropna()
Defense2020

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,Atlanta Hawks,43.3,90.7,0.478,11.9,32.7,0.363,31.4,58.0,0.542,...,15.0,119.7,33.4,24.0,7.8,5.1,0.543,12.7,74.9,0.233
1,Boston Celtics,38.4,87.1,0.441,11.8,34.8,0.340,26.6,52.3,0.508,...,15.2,107.3,35.4,23.0,8.3,5.6,0.509,13.5,77.4,0.215
2,Brooklyn Nets,41.4,92.9,0.446,12.2,34.8,0.350,29.2,58.1,0.503,...,12.8,112.3,37.3,24.5,6.4,4.5,0.511,11.0,77.8,0.187
3,Charlotte Hornets,41.7,87.5,0.476,12.4,35.1,0.352,29.3,52.5,0.558,...,14.4,109.6,31.8,23.8,6.6,4.1,0.546,13.1,74.4,0.159
4,Chicago Bulls,39.5,82.5,0.479,11.2,31.6,0.353,28.3,50.9,0.557,...,18.3,109.9,31.4,23.2,10.0,4.1,0.546,16.3,75.6,0.239
5,Cleveland Cavaliers,43.9,89.4,0.491,12.4,33.0,0.375,31.5,56.4,0.558,...,13.0,114.8,33.4,23.1,6.9,3.2,0.560,11.7,77.4,0.164
6,Dallas Mavericks,41.9,91.6,0.458,12.3,34.9,0.351,29.7,56.7,0.523,...,11.9,112.1,36.4,24.7,6.1,4.8,0.525,10.6,77.7,0.175
7,Denver Nuggets,40.0,86.5,0.463,12.1,33.9,0.358,27.9,52.6,0.530,...,14.3,109.2,33.4,26.7,8.0,4.6,0.533,12.9,76.8,0.198
8,Detroit Pistons,41.9,87.3,0.480,10.7,29.6,0.361,31.3,57.8,0.541,...,14.0,110.8,32.0,24.1,7.4,4.5,0.541,12.7,75.9,0.186
9,Golden State Warriors,42.2,88.6,0.476,13.5,34.8,0.389,28.7,53.8,0.533,...,15.6,115.0,32.9,25.6,8.2,4.6,0.553,13.7,76.4,0.193


In [61]:
## Duplicate the amount of times Clarkson opponents then delete his team and if he plays the team less than 3 times
newDefense2020 = pd.DataFrame(np.repeat(Defense2020.values,4,axis=0))
newDefense2020.columns = Defense2020.columns
newDefense2020

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,Atlanta Hawks,43.3,90.7,0.478,11.9,32.7,0.363,31.4,58,0.542,...,15,119.7,33.4,24,7.8,5.1,0.543,12.7,74.9,0.233
1,Atlanta Hawks,43.3,90.7,0.478,11.9,32.7,0.363,31.4,58,0.542,...,15,119.7,33.4,24,7.8,5.1,0.543,12.7,74.9,0.233
2,Atlanta Hawks,43.3,90.7,0.478,11.9,32.7,0.363,31.4,58,0.542,...,15,119.7,33.4,24,7.8,5.1,0.543,12.7,74.9,0.233
3,Atlanta Hawks,43.3,90.7,0.478,11.9,32.7,0.363,31.4,58,0.542,...,15,119.7,33.4,24,7.8,5.1,0.543,12.7,74.9,0.233
4,Boston Celtics,38.4,87.1,0.441,11.8,34.8,0.34,26.6,52.3,0.508,...,15.2,107.3,35.4,23,8.3,5.6,0.509,13.5,77.4,0.215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Utah Jazz,40.7,89.1,0.457,11,31,0.353,29.7,58.1,0.512,...,12.3,108.8,35.8,22.4,6.1,4.1,0.518,11.1,78.9,0.185
116,Washington Wizards,43.1,88.4,0.488,12.4,33.1,0.376,30.7,55.2,0.555,...,16.2,119.1,31.9,25,8,4.3,0.558,13.9,75.3,0.231
117,Washington Wizards,43.1,88.4,0.488,12.4,33.1,0.376,30.7,55.2,0.555,...,16.2,119.1,31.9,25,8,4.3,0.558,13.9,75.3,0.231
118,Washington Wizards,43.1,88.4,0.488,12.4,33.1,0.376,30.7,55.2,0.555,...,16.2,119.1,31.9,25,8,4.3,0.558,13.9,75.3,0.231


In [65]:
### Filter by opponent then combine after getting the correct amount of duplicates
df_Hawks = newDefense2020[newDefense2020['Team'] == 'Atlanta Hawks']
df_Hawks = df_Hawks.drop(df_Hawks.index[[[1,2,3]]])

### Filter by opponent then combine after getting the correct amount of duplicates
df_Celtics = newDefense2020[newDefense2020['Team'] == 'Boston Celtics']
df_Celtics = df_Celtics.drop(df_Celtics.index[[1,2]])

df_Nets = newDefense2020[newDefense2020['Team'] == 'Brooklyn Nets']
df_Nets = df_Nets.drop(df_Nets.index[[1,2]])

df_Bulls = newDefense2020[newDefense2020['Team'] == 'Chicago Bulls']
df_Bulls = df_Bulls.drop(df_Bulls.index[[[1,2,3]]])

df_Hornets = newDefense2020[newDefense2020['Team'] == 'Charlotte Hornets']
df_Hornets = df_Hornets.drop(df_Hornets.index[[1,2]])

df_Cavs = newDefense2020[newDefense2020['Team'] == 'Cleveland Cavaliers']
df_Cavs = df_Cavs.drop(df_Cavs.index[[[1,2,3]]])

df_Mavs = newDefense2020[newDefense2020['Team'] == 'Dallas Mavericks']
df_Mavs = df_Mavs.drop(df_Mavs.index[1])

df_Nuggets = newDefense2020[newDefense2020['Team'] == 'Denver Nuggets']
df_Nuggets = df_Nuggets.drop(df_Nuggets.index[1])

df_Pistons = newDefense2020[newDefense2020['Team'] == 'Detroit Pistons']
df_Pistons = df_Pistons.drop(df_Pistons.index[[1,2]])

df_Warriors = newDefense2020[newDefense2020['Team'] == 'Golden State Warriors']

df_Rockets = newDefense2020[newDefense2020['Team'] == 'Houston Rockets']
df_Rockets = df_Rockets.drop(df_Rockets.index[2])

df_Pacers = newDefense2020[newDefense2020['Team'] == 'Indiana Pacers']
df_Pacers = df_Pacers.drop(df_Pacers.index[[1,2]])

df_Clippers = newDefense2020[newDefense2020['Team'] == 'Los Angeles Clippers']
df_Clippers = df_Clippers.drop(df_Clippers.index[1])

df_Lakers = newDefense2020[newDefense2020['Team'] == 'Los Angeles Lakers']
df_Lakers = df_Lakers.drop(df_Lakers.index[1])

df_Grizz = newDefense2020[newDefense2020['Team'] == 'Memphis Grizzlies']

df_Heat = newDefense2020[newDefense2020['Team'] == 'Miami Heat']
df_Heat = df_Heat.drop(df_Heat.index[[1,2]])

df_Bucks = newDefense2020[newDefense2020['Team'] == 'Milwaukee Bucks']
df_Bucks = df_Bucks.drop(df_Bucks.index[[1,2]])

df_Wolves = newDefense2020[newDefense2020['Team'] == 'Minnesota Timberwolves']
df_Wolves = df_Wolves.drop(df_Wolves.index[2])

df_Pels = newDefense2020[newDefense2020['Team'] == 'New Orleans Pelicans']

df_Knicks = newDefense2020[newDefense2020['Team'] == 'New York Knicks']
df_Knicks = df_Knicks.drop(df_Knicks.index[[1,2]])

df_Thunder = newDefense2020[newDefense2020['Team'] == 'Oklahoma City Thunder']
df_Thunder = df_Thunder.drop(df_Thunder.index[1])

df_Magic = newDefense2020[newDefense2020['Team'] == 'Orlando Magic']
df_Magic = df_Magic.drop(df_Magic.index[[1,2]])

df_Sixers = newDefense2020[newDefense2020['Team'] == 'Philadelphia 76ers']
df_Sixers = df_Sixers.drop(df_Sixers.index[[1,2]])

df_Suns = newDefense2020[newDefense2020['Team'] == 'Phoenix Suns']
df_Suns = df_Suns.drop(df_Suns.index[[1,2]])

df_Blazers = newDefense2020[newDefense2020['Team'] == 'Portland Trail Blazers']
df_Blazers = df_Blazers.drop(df_Blazers.index[1])
                                            
df_Kings = newDefense2020[newDefense2020['Team'] == 'Sacramento Kings']
df_Kings = df_Kings.drop(df_Kings.index[1])

df_Spurs = newDefense2020[newDefense2020['Team'] == 'San Antonio Spurs']

df_Raptors = newDefense2020[newDefense2020['Team'] == 'Toronto Raptors']
df_Raptors = df_Raptors.drop(df_Raptors.index[[1,2]])

df_Wiz = newDefense2020[newDefense2020['Team'] == 'Washington Wizards']
df_Wiz = df_Wiz.drop(df_Wiz.index[[1,2]])

In [66]:
frames = [df_Hawks, df_Celtics, df_Nets, df_Bulls, df_Hornets, df_Cavs, df_Mavs, df_Nuggets, df_Pistons, df_Warriors,
         df_Rockets, df_Pacers, df_Clippers, df_Lakers, df_Grizz, df_Heat, df_Bucks, df_Wolves, df_Pels,
         df_Knicks, df_Thunder, df_Magic, df_Sixers, df_Suns, df_Blazers, df_Kings, df_Spurs, df_Raptors, df_Wiz ]

FinalDefense2020 = pd.concat(frames)

In [67]:
Joe_df2020 = pd.concat([Joe2020.reset_index(drop=True),FinalDefense2020.reset_index(drop=True)], axis=1)
Joe_df2020

,HomeAway,Opp,WinLoss,GS,MP,FG,FGA,FG%,3P,3PA,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,A,ATL,W,1,36,5,10,0.500,2,5,...,15,119.7,33.4,24,7.8,5.1,0.543,12.7,74.9,0.233
1,A,BOS,W,0,30,3,7,0.429,3,6,...,15.2,107.3,35.4,23,8.3,5.6,0.509,13.5,77.4,0.215
2,H,BOS,L,0,26,1,3,0.333,0,2,...,15.2,107.3,35.4,23,8.3,5.6,0.509,13.5,77.4,0.215
3,A,BRK,W,1,30,10,14,0.714,6,8,...,12.8,112.3,37.3,24.5,6.4,4.5,0.511,11,77.8,0.187
4,H,BRK,W,0,32,0,6,0.000,0,4,...,12.8,112.3,37.3,24.5,6.4,4.5,0.511,11,77.8,0.187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,A,SAS,L,1,30,3,8,0.375,1,4,...,13.2,115.2,35.6,24.7,7.3,5.5,0.542,11.7,79.2,0.193
68,H,TOR,L,0,32,8,12,0.667,4,8,...,16.8,106.5,35.9,25.2,8.8,5,0.502,14.6,76.7,0.202
69,A,TOR,L,0,26,2,5,0.400,2,3,...,16.8,106.5,35.9,25.2,8.8,5,0.502,14.6,76.7,0.202
70,H,WAS,W,0,27,3,5,0.600,2,4,...,16.2,119.1,31.9,25,8,4.3,0.558,13.9,75.3,0.231


In [68]:
### Repeat for previous 2 seasons for the training data

## upload Jordan Clarkson 2019 data

address = '/Users/raytrey5/Desktop/IST_718_Project/Joe_Ingles_2019_GameLog.csv'
Joe2019 = pd.read_csv(address)
## drop NAs columns 
Joe2019 = Joe2019.drop(columns=Joe2019.columns[24:30])
Joe2019

,HomeAway,Opp,WinLoss,GS,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Three+3s
0,A,ATL,L,1,33,5,8,0.625,3,5,...,0,2,2,7,2,0,3,3,13,1
1,H,ATL,W,1,32,7,12,0.583,3,6,...,1,6,7,5,0,1,2,2,19,1
2,A,BOS,W,1,30,3,6,0.500,1,4,...,0,3,3,6,0,0,3,3,7,0
3,H,BOS,W,1,35,10,14,0.714,5,9,...,0,5,5,7,1,1,1,3,27,1
4,H,BRK,W,1,30,1,5,0.200,0,3,...,0,6,6,4,1,0,3,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,H,SAS,W,1,27,5,9,0.556,3,6,...,0,3,3,7,3,0,2,1,13,1
78,A,TOR,L,1,34,0,5,0.000,0,5,...,0,6,6,6,2,0,1,5,2,0
79,H,TOR,L,1,32,3,13,0.231,2,6,...,1,1,2,2,1,0,2,2,9,0
80,H,WAS,W,1,36,7,14,0.500,3,8,...,1,4,5,10,0,0,1,1,18,1


In [69]:
# Fill in Nas with mean... Free throw percentage...There were
# games he did not attempt a Free throw thus it was NA

column_means = Joe2019.mean()
Joe2019 = Joe2019.fillna(column_means)
## check to see if there's no NAs
Joe2019.isnull().values.any()

False

In [70]:
## upload 2019 Defensive metrics

address = '/Users/raytrey5/Desktop/IST_718_Project/OpponentStatsPerGame2019.csv'
Defense2019 = pd.read_csv(address)
Defense2019 = Defense2019.dropna()
Defense2019

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,Atlanta Hawks,42.8,90.5,0.473,12.4,34.4,0.359,30.4,56.1,0.543,...,15.0,119.4,34.5,25.8,8.2,5.1,0.541,12.8,76.4,0.237
1,Boston Celtics,39.5,88.1,0.448,11.5,33.5,0.344,28.0,54.6,0.513,...,15.1,108.0,34.7,26.3,8.6,5.3,0.514,13.4,77.0,0.198
2,Brooklyn Nets,42.2,92.5,0.456,10.3,30.3,0.341,31.9,62.1,0.513,...,13.5,112.3,35.6,23.8,6.6,4.1,0.512,11.6,76.4,0.190
3,Charlotte Hornets,41.9,89.2,0.470,12.1,33.5,0.361,29.8,55.6,0.536,...,13.5,111.8,33.9,23.2,7.2,4.9,0.538,12.1,77.1,0.179
4,Chicago Bulls,42.2,89.2,0.473,12.1,33.0,0.365,30.2,56.2,0.537,...,13.3,113.4,34.1,21.9,7.4,4.3,0.541,11.9,77.3,0.188
5,Cleveland Cavaliers,43.2,87.2,0.495,12.0,31.6,0.380,31.2,55.6,0.561,...,12.5,114.1,31.9,20.7,6.5,2.4,0.564,11.5,77.0,0.181
6,Dallas Mavericks,40.9,89.2,0.459,11.2,32.5,0.344,29.7,56.7,0.525,...,13.0,110.1,35.2,23.4,6.5,4.3,0.522,11.6,77.5,0.191
7,Denver Nuggets,39.6,86.4,0.458,10.8,31.9,0.339,28.8,54.5,0.528,...,13.5,106.7,34.5,27.4,7.7,4.4,0.521,12.3,78.0,0.194
8,Detroit Pistons,39.9,85.0,0.469,9.6,27.8,0.345,30.3,57.2,0.530,...,14.1,107.3,33.6,22.5,6.9,4.0,0.526,12.8,78.7,0.211
9,Golden State Warriors,40.4,91.1,0.444,11.6,33.6,0.347,28.8,57.5,0.500,...,13.5,111.2,36.5,29.4,7.6,6.4,0.508,11.7,77.1,0.205


In [71]:
## Duplicate the amount of times Clarkson opponents then delete his team and if he plays the team less than 3 times
newDefense2019 = pd.DataFrame(np.repeat(Defense2019.values,4,axis=0))
newDefense2019.columns = Defense2019.columns
newDefense2019

,Team,OppFG,OppFGA,OppFGP,Opp3P,Opp3PA,Opp3PPerc,Opp2P,Opp2PA,Opp2PPerc,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,Atlanta Hawks,42.8,90.5,0.473,12.4,34.4,0.359,30.4,56.1,0.543,...,15,119.4,34.5,25.8,8.2,5.1,0.541,12.8,76.4,0.237
1,Atlanta Hawks,42.8,90.5,0.473,12.4,34.4,0.359,30.4,56.1,0.543,...,15,119.4,34.5,25.8,8.2,5.1,0.541,12.8,76.4,0.237
2,Atlanta Hawks,42.8,90.5,0.473,12.4,34.4,0.359,30.4,56.1,0.543,...,15,119.4,34.5,25.8,8.2,5.1,0.541,12.8,76.4,0.237
3,Atlanta Hawks,42.8,90.5,0.473,12.4,34.4,0.359,30.4,56.1,0.543,...,15,119.4,34.5,25.8,8.2,5.1,0.541,12.8,76.4,0.237
4,Boston Celtics,39.5,88.1,0.448,11.5,33.5,0.344,28,54.6,0.513,...,15.1,108,34.7,26.3,8.6,5.3,0.514,13.4,77,0.198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Utah Jazz,40,88.5,0.452,9.8,27.7,0.355,30.1,60.7,0.496,...,13.9,106.5,36.4,26,8.1,5.9,0.507,12.4,80.3,0.189
116,Washington Wizards,43.4,90.5,0.48,12.1,32.8,0.37,31.3,57.7,0.542,...,15.8,116.9,32.7,26.3,8.3,4.6,0.546,13.5,74.1,0.199
117,Washington Wizards,43.4,90.5,0.48,12.1,32.8,0.37,31.3,57.7,0.542,...,15.8,116.9,32.7,26.3,8.3,4.6,0.546,13.5,74.1,0.199
118,Washington Wizards,43.4,90.5,0.48,12.1,32.8,0.37,31.3,57.7,0.542,...,15.8,116.9,32.7,26.3,8.3,4.6,0.546,13.5,74.1,0.199


In [74]:
print(Joe2019[50:])

   HomeAway  Opp WinLoss  GS  MP  FG  FGA    FG%  3P  3PA  ...  ORB  DRB  TRB  \
50        A  NOP       W   1  38   4   12  0.333   1    7  ...    1    7    8   
51        H  NOP       L   1  34   2    7  0.286   2    6  ...    0    4    4   
52        A  NOP       W   1  29   2   10  0.200   0    4  ...    0    4    4   
53        A  NYK       W   1  24   6    9  0.667   6    9  ...    0    5    5   
54        H  NYK       W   1  21   1    5  0.200   0    3  ...    0    1    1   
55        H  OKC       L   1  30   2    9  0.222   1    7  ...    1    3    4   
56        A  OKC       L   1  46   5   10  0.500   5    9  ...    1    4    5   
57        H  OKC       L   1  34   5   12  0.417   3    8  ...    0    4    4   
58        A  OKC       L   1  25   3    7  0.429   0    2  ...    1    2    3   
59        H  ORL       W   1  35   6   15  0.400   4    8  ...    0    2    2   
60        A  ORL       L   1  33   2   11  0.182   2    6  ...    1    4    5   
61        H  PHI       L   1

In [77]:
### Filter by opponent then combine after getting the correct amount of duplicates
df_Hawks = newDefense2019[newDefense2019['Team'] == 'Atlanta Hawks']
df_Hawks = df_Hawks.drop(df_Hawks.index[[1,2]])

### Filter by opponent then combine after getting the correct amount of duplicates
df_Celtics = newDefense2019[newDefense2019['Team'] == 'Boston Celtics']
df_Celtics = df_Celtics.drop(df_Celtics.index[[1,2]])

df_Nets = newDefense2019[newDefense2019['Team'] == 'Brooklyn Nets']
df_Nets = df_Nets.drop(df_Nets.index[[1,2]])

df_Bulls = newDefense2019[newDefense2019['Team'] == 'Chicago Bulls']
df_Bulls = df_Bulls.drop(df_Bulls.index[[1,2]])

df_Hornets = newDefense2019[newDefense2019['Team'] == 'Charlotte Hornets']
df_Hornets = df_Hornets.drop(df_Hornets.index[[1,2]])

df_Cavs = newDefense2019[newDefense2019['Team'] == 'Cleveland Cavaliers']
df_Cavs = df_Cavs.drop(df_Cavs.index[[1,2]])

df_Mavs = newDefense2019[newDefense2019['Team'] == 'Dallas Mavericks']

df_Nuggets = newDefense2019[newDefense2019['Team'] == 'Denver Nuggets']

df_Pistons = newDefense2019[newDefense2019['Team'] == 'Detroit Pistons']
df_Pistons = df_Pistons.drop(df_Pistons.index[[1,2]])

df_Warriors = newDefense2019[newDefense2019['Team'] == 'Golden State Warriors']
df_Warriors = df_Warriors.drop(df_Warriors.index[1])                                        

df_Rockets = newDefense2019[newDefense2019['Team'] == 'Houston Rockets']

df_Pacers = newDefense2019[newDefense2019['Team'] == 'Indiana Pacers']
df_Pacers = df_Pacers.drop(df_Pacers.index[[1,2]])

df_Clippers = newDefense2019[newDefense2019['Team'] == 'Los Angeles Clippers']
df_Clippers = df_Clippers.drop(df_Clippers.index[1])

df_Lakers = newDefense2019[newDefense2019['Team'] == 'Los Angeles Lakers']

df_Grizz = newDefense2019[newDefense2019['Team'] == 'Memphis Grizzlies']

df_Heat = newDefense2019[newDefense2019['Team'] == 'Miami Heat']
df_Heat = df_Heat.drop(df_Heat.index[[1,2]])

df_Bucks = newDefense2019[newDefense2019['Team'] == 'Milwaukee Bucks']
df_Bucks = df_Bucks.drop(df_Bucks.index[[1,2]])

df_Wolves = newDefense2019[newDefense2019['Team'] == 'Minnesota Timberwolves']

df_Pels = newDefense2019[newDefense2019['Team'] == 'New Orleans Pelicans']
df_Pels = df_Pels.drop(df_Pels.index[1])

df_Knicks = newDefense2019[newDefense2019['Team'] == 'New York Knicks']
df_Knicks = df_Knicks.drop(df_Knicks.index[[1,2]])

df_Thunder = newDefense2019[newDefense2019['Team'] == 'Oklahoma City Thunder']

df_Magic = newDefense2019[newDefense2019['Team'] == 'Orlando Magic']
df_Magic = df_Magic.drop(df_Magic.index[[1,2]])

df_Sixers = newDefense2019[newDefense2019['Team'] == 'Philadelphia 76ers']
df_Sixers = df_Sixers.drop(df_Sixers.index[[1,2]])

df_Suns = newDefense2019[newDefense2019['Team'] == 'Phoenix Suns']

df_Blazers = newDefense2019[newDefense2019['Team'] == 'Portland Trail Blazers']

                                            
df_Kings = newDefense2019[newDefense2019['Team'] == 'Sacramento Kings']


df_Spurs = newDefense2019[newDefense2019['Team'] == 'San Antonio Spurs']
df_Spurs = df_Spurs.drop(df_Spurs.index[1])                                        

df_Raptors = newDefense2019[newDefense2019['Team'] == 'Toronto Raptors']
df_Raptors = df_Raptors.drop(df_Raptors.index[[1,2]])

df_Wiz = newDefense2019[newDefense2019['Team'] == 'Washington Wizards']
df_Wiz = df_Wiz.drop(df_Wiz.index[[1,2]])


In [78]:
frames = [df_Hawks, df_Celtics, df_Nets, df_Bulls, df_Hornets, df_Cavs, df_Mavs, df_Nuggets, df_Pistons, df_Warriors,
         df_Rockets, df_Pacers, df_Clippers, df_Lakers, df_Grizz, df_Heat, df_Bucks, df_Wolves, df_Pels,
         df_Knicks, df_Thunder, df_Magic, df_Sixers, df_Suns, df_Blazers, df_Kings, df_Spurs, df_Raptors, df_Wiz ]

FinalDefense2019 = pd.concat(frames)

In [79]:
Joe_df2019 = pd.concat([Joe2019.reset_index(drop=True),FinalDefense2019.reset_index(drop=True)], axis=1)
Joe_df2019

,HomeAway,Opp,WinLoss,GS,MP,FG,FGA,FG%,3P,3PA,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,A,ATL,L,1,33,5,8,0.625,3,5,...,15,119.4,34.5,25.8,8.2,5.1,0.541,12.8,76.4,0.237
1,H,ATL,W,1,32,7,12,0.583,3,6,...,15,119.4,34.5,25.8,8.2,5.1,0.541,12.8,76.4,0.237
2,A,BOS,W,1,30,3,6,0.500,1,4,...,15.1,108,34.7,26.3,8.6,5.3,0.514,13.4,77,0.198
3,H,BOS,W,1,35,10,14,0.714,5,9,...,15.1,108,34.7,26.3,8.6,5.3,0.514,13.4,77,0.198
4,H,BRK,W,1,30,1,5,0.200,0,3,...,13.5,112.3,35.6,23.8,6.6,4.1,0.512,11.6,76.4,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,H,SAS,W,1,27,5,9,0.556,3,6,...,12.2,110,35.5,24.5,6.1,4.7,0.528,11,79.4,0.17
78,A,TOR,L,1,34,0,5,0.000,0,5,...,14.9,108.4,35.6,25.4,8.3,5.3,0.509,13.1,77.1,0.19
79,H,TOR,L,1,32,3,13,0.231,2,6,...,14.9,108.4,35.6,25.4,8.3,5.3,0.509,13.1,77.1,0.19
80,H,WAS,W,1,36,7,14,0.500,3,8,...,15.8,116.9,32.7,26.3,8.3,4.6,0.546,13.5,74.1,0.199


In [80]:
### Get Final Training data 

frames = [Joe_df2019, Joe_df2020, Joe_df2021]

data = pd.concat(frames)
data

,HomeAway,Opp,WinLoss,GS,MP,FG,FGA,FG%,3P,3PA,...,OppTOV,OppPTS,Team_DRB,Team_AST,Team_STL,Team_BLK,OppEffFGperc,OppTOVperc,OppDRBperc,FT/FGA
0,A,ATL,L,1.0,33.0,5.0,8.0,0.625,3.0,5.0,...,15,119.4,34.5,25.8,8.2,5.1,0.541,12.8,76.4,0.237
1,H,ATL,W,1.0,32.0,7.0,12.0,0.583,3.0,6.0,...,15,119.4,34.5,25.8,8.2,5.1,0.541,12.8,76.4,0.237
2,A,BOS,W,1.0,30.0,3.0,6.0,0.500,1.0,4.0,...,15.1,108,34.7,26.3,8.6,5.3,0.514,13.4,77,0.198
3,H,BOS,W,1.0,35.0,10.0,14.0,0.714,5.0,9.0,...,15.1,108,34.7,26.3,8.6,5.3,0.514,13.4,77,0.198
4,H,BRK,W,1.0,30.0,1.0,5.0,0.200,0.0,3.0,...,13.5,112.3,35.6,23.8,6.6,4.1,0.512,11.6,76.4,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,A,SAS,W,0.0,21.0,5.0,9.0,0.556,2.0,6.0,...,13.4,112.2,34.6,24.4,7,5.1,0.541,11.8,77.3,0.174
60,H,TOR,W,1.0,34.0,4.0,8.0,0.500,4.0,7.0,...,13.4,112.2,34.6,24.4,7,5.1,0.541,11.8,77.3,0.174
61,A,TOR,W,0.0,33.0,5.0,6.0,0.833,5.0,6.0,...,13.4,112.2,34.6,24.4,7,5.1,0.541,11.8,77.3,0.174
62,H,WAS,L,1.0,31.0,6.0,16.0,0.375,3.0,9.0,...,16.2,111.5,32.1,24.1,8.6,5.4,0.543,14.4,76.3,0.234


In [81]:
data.to_csv(r'/Users/raytrey5/Desktop/IST_718_Project/JoeIngles_data.csv', index = False)